Just vote from these models below:
- **EfficientNet v1 by pytorch:**  
  Eye -- EfficientNet Pytorch[LB 0.777] written by Chanhu  
  https://www.kaggle.com/chanhu/eye-efficientnet-pytorch-lb-0-777
    
- **EfficientNet v2 by fastai: **   
  Starter kernel for > 0.79  
  https://www.kaggle.com/drhabib/starter-kernel-for-0-79

- **Resnet101 by pytorch:**  
  very simple pytorch training [0.59+] written by Abhishek Thakur  
  https://www.kaggle.com/abhishek/very-simple-pytorch-training-0-59

- **DenseNet by keras: **   
  APTOS 2019: DenseNet Keras Starter  
  https://www.kaggle.com/xhlulu/aptos-2019-densenet-keras-starter
    
- **Resnet50 by fastai:  **  
  fast.ai starter with ResNet 50 written by Anna Novikova  
  https://www.kaggle.com/demonplus/fast-ai-starter-with-resnet-50


And about the voting code, I learned it from here and did some changes:  
- **aptos_Vote**    
https://www.kaggle.com/ahoukang/aptos-vote

# EfficientNet_v2

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
%matplotlib inline  
from sklearn.model_selection import StratifiedKFold
from joblib import load, dump
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from torchvision import models as md
from torch import nn
from torch.nn import functional as F
import re
import math
import collections
from functools import partial
from torch.utils import model_zoo
from sklearn import metrics
from collections import Counter
import json

In [ ]:
package_path = '../input/efficientnet/efficientnet-pytorch/EfficientNet-PyTorch/'
sys.path.append(package_path)

from efficientnet_pytorch import EfficientNet

In [ ]:
#making model
# md_ef = EfficientNet.from_pretrained('efficientnet-b5', num_classes=1)

num_classes = 1

md_ef = EfficientNet.from_name('efficientnet-b5')
md_ef.load_state_dict(torch.load('../input/efficientnet-pytorch/efficientnet-b5-586e6cc6.pth'))
in_features = md_ef._fc.in_features
md_ef._fc = nn.Linear(in_features, num_classes)
md_ef.cuda()

In [ ]:
#copying weighst to the local directory 
!mkdir models
!cp '../input/kaggle-public/abcdef.pth' 'models'

In [ ]:
def get_df():
    base_image_dir = os.path.join('..', 'input/aptos2019-blindness-detection/')
    train_dir = os.path.join(base_image_dir,'train_images/')
    df = pd.read_csv(os.path.join(base_image_dir, 'train.csv'))
    df['path'] = df['id_code'].map(lambda x: os.path.join(train_dir,'{}.png'.format(x)))
    df = df.drop(columns=['id_code'])
    df = df.sample(frac=1).reset_index(drop=True) #shuffle dataframe
    test_df = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
    return df, test_df

df, test_df = get_df()

In [ ]:
#you can play around with tfms and image sizes
bs = 64
sz = 224
tfms = get_transforms(do_flip=True,flip_vert=True)

In [ ]:
data = (ImageList.from_df(df=df,path='./',cols='path') 
        .split_by_rand_pct(0.2) 
        .label_from_df(cols='diagnosis',label_cls=FloatList) 
        .transform(tfms,size=sz,resize_method=ResizeMethod.SQUISH,padding_mode='zeros') 
        .databunch(bs=bs,num_workers=4) 
        .normalize(imagenet_stats)  
       )

In [ ]:
def qk(y_pred, y):
    return torch.tensor(cohen_kappa_score(torch.round(y_pred), y, weights='quadratic'), device='cuda:0')

In [ ]:
learn = Learner(data, 
                md_ef, 
                metrics = [qk], 
                model_dir="models").to_fp16()

learn.data.add_test(ImageList.from_df(test_df,
                                      '../input/aptos2019-blindness-detection',
                                      folder='test_images',
                                      suffix='.png'))



Try to fit model here:
> learn.fit_one_cycle(10, 1e-3)

In [ ]:
learn.load('abcdef');

In [ ]:
#https://www.kaggle.com/abhishek/optimizer-for-quadratic-weighted-kappa
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4

        ll = metrics.cohen_kappa_score(y, X_p, weights='quadratic')
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')
        print(-loss_partial(self.coef_['x']))

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4
        return X_p

    def coefficients(self):
        return self.coef_['x']

In [ ]:
def run_subm(learn=learn, coefficients=[0.5, 1.5, 2.5, 3.5]):
    opt = OptimizedRounder()
    preds,y = learn.get_preds(DatasetType.Test)
    tst_pred = opt.predict(preds, coefficients)
    tst_pred = tst_pred.astype(int)
    return tst_pred

In [ ]:
preds1 = run_subm()

# Resnet101

In [ ]:
import sys
package_dir = "../input/pretrained-models/pretrained-models/pretrained-models.pytorch-master/"
sys.path.insert(0, package_dir)
import numpy as np
import pandas as pd
import torchvision
import torch.nn as nn
from tqdm import tqdm
from PIL import Image, ImageFile
from torch.utils.data import Dataset
import torch
from torchvision import transforms
import os
import pretrainedmodels

device = torch.device("cuda:0")
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
class RetinopathyDatasetTest(Dataset):
    def __init__(self, csv_file, transform):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join('../input/aptos2019-blindness-detection/test_images', self.data.loc[idx, 'id_code'] + '.png')
        image = Image.open(img_name)
        image = self.transform(image)
        return {'image': image}

In [ ]:
model = pretrainedmodels.__dict__['resnet101'](pretrained=None)

model.avg_pool = nn.AdaptiveAvgPool2d(1)
model.last_linear = nn.Sequential(
                          nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                          nn.Dropout(p=0.25),
                          nn.Linear(in_features=2048, out_features=2048, bias=True),
                          nn.ReLU(),
                          nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                          nn.Dropout(p=0.5),
                          nn.Linear(in_features=2048, out_features=1, bias=True),
                         )
model.load_state_dict(torch.load("../input/mmmodel/model.bin"))
model = model.to(device)

In [ ]:
for param in model.parameters():
    param.requires_grad = False

model.eval()

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_dataset = RetinopathyDatasetTest(csv_file='../input/aptos2019-blindness-detection/sample_submission.csv',
                                      transform=test_transform)

In [ ]:
def tta(test_dataset, batch_size=32, shuffle=False, num_workers=4):
    test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
    test_preds = np.zeros((len(test_dataset), 1))
    tk = tqdm(test_data_loader)
    
    for i, x_batch in enumerate(tk):
        x_batch = x_batch['image']
        pred = model(x_batch.to(device))
        test_preds[i*32:(i+1)*32] = pred.detach().cpu().squeeze().numpy().ravel().reshape(-1, 1)
    
    return test_preds

In [ ]:
for i in range(1, 6):
    locals()['test_preds{}'.format(i)] = tta(test_dataset) 

In [ ]:
preds2 = (test_preds1 + test_preds2 + test_preds3 + test_preds4 + test_preds5) / 5.0

In [ ]:
coef = [0.5, 1.5, 2.5, 3.5]

for i, pred in enumerate(preds2):
    if pred < coef[0]:
        preds2[i] = 0
    elif pred >= coef[0] and pred < coef[1]:
        preds2[i] = 1
    elif pred >= coef[1] and pred < coef[2]:
        preds2[i] = 2
    elif pred >= coef[2] and pred < coef[3]:
        preds2[i] = 3
    else:
        preds2[i] = 4

# EfficientNet v1

In [ ]:
import cv2
import matplotlib.pyplot as plt
from os.path import isfile
# import torch.nn.init as init
import torch
import torch.nn as nn
import numpy as np
import pandas as pd 
import os
from PIL import Image
# from PIL import ImageFilter
# print(os.listdir("../input"))
# from sklearn.model_selection import train_test_split, StratifiedKFold
from torch.utils.data import Dataset
from torchvision import transforms
# from torch.optim import Adam, SGD, RMSprop
import time
# from torch.autograd import Variable
# import torch.functional as F
from tqdm import tqdm
# from sklearn import metrics
# import urllib
# import pickle
# from torchvision import models
# import seaborn as sns
import random
import sys

In [ ]:
package_path = '../input/efficientnet/efficientnet-pytorch/EfficientNet-PyTorch/'
sys.path.append(package_path)

In [ ]:
## EfficientNet是谷歌开源的新模型。
from efficientnet_pytorch import EfficientNet

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
seed_everything(1234)
TTA         = 5
num_classes = 1
IMG_SIZE    = 384

In [ ]:
test = '../input/aptos2019-blindness-detection/test_images/'

In [ ]:
def expand_path(p):
    p = str(p)
    if isfile(test + p + ".png"):
        return test + (p + ".png")
    return p

In [ ]:
def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img

In [ ]:
class MyDataset(Dataset):
    
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        
        label = self.df.diagnosis.values[idx]
        label = np.expand_dims(label, -1)
        
        p = self.df.id_code.values[idx]
        p_path = expand_path(p)
        image = cv2.imread(p_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = crop_image_from_gray(image)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , 30) ,-4 ,128)
        image = transforms.ToPILImage()(image)
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [ ]:
test_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation((-120, 120)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

testset        = MyDataset(pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv'), 
                 transform=test_transform)
test_loader    = torch.utils.data.DataLoader(testset, batch_size=16, shuffle=False)

In [ ]:
model = EfficientNet.from_name('efficientnet-b0')
in_features = model._fc.in_features
model._fc = nn.Linear(in_features, num_classes)
model.load_state_dict(torch.load('../input/enet0802/weight_best_0802.pt'))
model.cuda()

In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
sample = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')

In [ ]:
%%time
test_pred = np.zeros((len(sample), 1))
model.eval()

for _ in range(TTA):
    with torch.no_grad():
        for i, data in tqdm(enumerate(test_loader)):
            images, _ = data
            images = images.cuda()
            pred = model(images)
            test_pred[i * 16:(i + 1) * 16] += pred.detach().cpu().squeeze().numpy().reshape(-1, 1)
        
output = test_pred / TTA

In [ ]:
coef = [0.57, 1.37, 2.57, 3.57]

for i, pred in enumerate(output):
    if pred < coef[0]:
        output[i] = 0
    elif pred >= coef[0] and pred < coef[1]:
        output[i] = 1
    elif pred >= coef[1] and pred < coef[2]:
        output[i] = 2
    elif pred >= coef[2] and pred < coef[3]:
        output[i] = 3
    else:
        output[i] = 4
        
preds3 = output

# DesNet v2

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import (Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D,
                          BatchNormalization, Input, Conv2D, GlobalAveragePooling2D,concatenate,Concatenate)
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam 
from keras import backend as K
from keras.models import load_model
import keras
from keras.models import Model

import matplotlib.pyplot as plt
import skimage.io
from skimage.transform import resize
from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image, ImageOps
import cv2
from sklearn.utils import class_weight, shuffle
from keras.losses import binary_crossentropy, categorical_crossentropy
#from keras.applications.resnet50 import preprocess_input
from keras.applications.densenet import DenseNet121,DenseNet169
import keras.backend as K
import tensorflow as tf
from sklearn.metrics import f1_score, fbeta_score, cohen_kappa_score
from keras.utils import Sequence
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import imgaug as ia

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../input"))

WORKERS = 2
CHANNEL = 3

import warnings
warnings.filterwarnings("ignore")
SIZE = 300
NUM_CLASSES = 5

In [ ]:
df_test = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')

In [ ]:
def create_model(input_shape, n_out):
    input_tensor = Input(shape=input_shape)
    base_model = DenseNet121(include_top=False,
                   weights=None,
                   input_tensor=input_tensor)
    base_model.load_weights("../input/densenet-keras/DenseNet-BC-121-32-no-top.h5")
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    final_output = Dense(n_out, activation='softmax', name='final_output')(x)
    model = Model(input_tensor, final_output) 
    return model

In [ ]:
submit = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')

model = load_model('../input/densenet_v2_0817_2/densenet_v2_0817_2.h5')
preds4 = []

In [ ]:
# reference:https://www.kaggle.com/CVxTz/cnn-starter-nasnet-mobile-0-9709-lb 
for i, name in tqdm(enumerate(submit['id_code']), total=len(submit)):
    path = os.path.join('../input/aptos2019-blindness-detection/test_images/', name+'.png')
    image = cv2.imread(path)
    image = cv2.resize(image, (SIZE, SIZE))
    X = np.array((image[np.newaxis])/255)
    score_predict=((model.predict(X).ravel()*model.predict(X[:, ::-1, :, :]).ravel()*model.predict(X[:, ::-1, ::-1, :]).ravel()*model.predict(X[:, :, ::-1, :]).ravel())**0.25).tolist()
    label_predict = np.argmax(score_predict)
    preds4.append(label_predict)

preds4 = np.array(preds4) 

# ResNet50

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai import *
from fastai.vision import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy as sp

from functools import partial
from sklearn import metrics
from collections import Counter
from fastai.callbacks import *

import PIL
import cv2

In [ ]:
# Set seed for all
def seed_everything(seed=1358):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything()

In [ ]:
PATH = Path('../input/aptos2019-blindness-detection')

df = pd.read_csv(PATH/'train.csv')

In [ ]:
!ls ../input/resnet50/

In [ ]:
# copy pretrained weights for resnet50 to the folder fastai will search by default
Path('/tmp/.cache/torch/checkpoints/').mkdir(exist_ok=True, parents=True)
!cp '../input/resnet50/resnet50.pth' '/tmp/.cache/torch/checkpoints/resnet50-19c8e357.pth'

#copying weighst to the local directory 
!mkdir kaggle
!cp '../input/resnet50-0823/resnet50_0823.pth' '/kaggle/'

In [ ]:
def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

In [ ]:
IMG_SIZE = 512

def _load_format(path, convert_mode, after_open)->Image:
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0), 10) ,-4 ,128)
                    
    return Image(pil2tensor(image, np.float32).div_(255)) #return fastai Image format

vision.data.open_image = _load_format
    
src = (
    ImageList.from_df(df,PATH,folder='train_images',suffix='.png')
        .split_by_rand_pct(0.2, seed=42)
        .label_from_df(cols='diagnosis',label_cls=FloatList)    
    )

In [ ]:
tfms = get_transforms(do_flip=True, flip_vert=True, max_rotate=0.10, max_zoom=1.3, max_warp=0.0, max_lighting=0.2)

In [ ]:
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4

        ll = metrics.cohen_kappa_score(y, X_p, weights='quadratic')
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')
        print(-loss_partial(self.coef_['x']))

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4
        return X_p

    def coefficients(self):
        return self.coef_['x']

In [ ]:
sample_df = pd.read_csv(PATH/'sample_submission.csv')
sample_df.head()

In [ ]:
# Definition of Quadratic Kappa
from sklearn.metrics import cohen_kappa_score
def quadratic_kappa(y_hat, y):
    return torch.tensor(cohen_kappa_score(torch.round(y_hat), y, weights='quadratic'),device='cuda:0')

data = (
    src.transform(tfms,size=224)
    .databunch()
    .normalize(imagenet_stats)
)

learn = cnn_learner(data, base_arch=models.resnet50 ,metrics=[quadratic_kappa],model_dir='/kaggle',pretrained=True)
learn.data.add_test(ImageList.from_df(sample_df,PATH,folder='test_images',suffix='.png'))
learn.load('resnet50_0823')

In [ ]:
preds,y = learn.get_preds(DatasetType.Test)

In [ ]:
optR = OptimizedRounder()

coefficients = [0.563203, 1.400153, 2.43412,  3.439786]
test_predictions = optR.predict(preds, coefficients)

In [ ]:
preds5 = test_predictions.astype(int)

# Ensemble

In [ ]:
preds1 = preds1.flatten()
preds2 = preds2.flatten()
preds3 = preds3.flatten()
preds4 = preds4.flatten()
preds5 = preds5.flatten()

In [ ]:
score = [0.783, 0.750, 0.783, 0.741, 0.738]
weight = [0.23, 0.195, 0.23, 0.175, 0.170]
predsData = weight[0]*preds1 + weight[1]*preds2 + weight[2]*preds3 + weight[3]*preds4 + weight[4]*preds5

coef = [0.5, 1.5, 2.5, 3.5]
for i, pred in enumerate(predsData):
    if pred < coef[0]:
        predsData[i] = 0
    elif pred >= coef[0] and pred < coef[1]:
        predsData[i] = 1
    elif pred >= coef[1] and pred < coef[2]:
        predsData[i] = 2
    elif pred >= coef[2] and pred < coef[3]:
        predsData[i] = 3
    else:
        predsData[i] = 4
submission = pd.read_csv("../input/aptos2019-blindness-detection/sample_submission.csv")
submission.diagnosis = predsData.astype(int)
submission.to_csv('submission.csv', index=False)